In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
import tifffile
import numpy as np
from skimage.measure import label, regionprops

In [ ]:
# Clase para cargar las imágenes, las máscaras y las etiquetas
class ImageDataset:
    def __init__(self, image_paths, labels_paths, class_paths):
        if len(image_paths) != len(class_paths):
            raise ValueError("Las rutas de imágenes y máscaras deben coincidir.")
        self.image_paths = image_paths
        self.labels_paths = labels_paths
        self.class_paths = class_paths

    def load_image(self, index=0):
        return tifffile.imread(self.image_paths[index])

    def load_labels(self, index=0):
        return tifffile.imread(self.labels_paths[index])

    def load_class(self, index=0):
        return tifffile.imread(self.class_paths[index])

    def __len__(self):
        return len(self.image_paths)

# Función para verificar si una región toca el borde de la imagen y exlcuir las celulas
def _touches_border_3d(bbox, shape_zyx, border_margin=0):
    min_z, min_y, min_x, max_z, max_y, max_x = bbox
    Z, Y, X = shape_zyx
    if min_z <= border_margin or min_y <= border_margin or min_x <= border_margin:
        return True
    if max_z >= (Z - border_margin) or max_y >= (Y - border_margin) or max_x >= (X - border_margin):
        return True
    return False

# Función para guardar los parches generados
def save_bbox_patches(image3d, labels_3d, classes_3d, output_dir, border_margin=0, min_voxels=0):
    os.makedirs(output_dir, exist_ok=True)

    # Identificamos las clases presentes, excluyendo el fondo (valor 0)
    classes_present = np.unique(classes_3d)
    classes_present = classes_present[classes_present != 0]

    # Crear carpeta para cada clase
    for class_value in classes_present:
        print(f"Procesando clase: {class_value}")
        label_folder = os.path.join(output_dir, f"label_{class_value}")
        os.makedirs(label_folder, exist_ok=True)

        # Obtener las propiedades de las regiones
        props_image = regionprops(labels_3d, intensity_image=image3d)
        props_class = regionprops(labels_3d, intensity_image=classes_3d)
        idx = 0

        # Iteramos sobre cada región y clase
        for region, region_cls in zip(props_image, props_class):
            label = region.label
            if min_voxels > 0 and region.area < min_voxels:
                continue
            if _touches_border_3d(region.bbox, image3d.shape, border_margin=border_margin):
                continue

            # Obtenemos las intensidades y la clase de la región
            intensities = region_cls.intensity_image
            current_class = np.amax(np.unique(intensities[intensities != 0]))

            # Crear la carpeta para la clase específica si no existe
            label_folder = os.path.join(output_dir, f"label_{current_class}")
            os.makedirs(label_folder, exist_ok=True)

            # Extraemos la imagen de la región
            img = region.image_intensity.astype(np.float32)
            mask = region.image_filled.astype(np.uint8)
            masked_img = np.zeros_like(img)
            masked_img[mask > 0] = img[mask > 0]

            # Guardamos la imagen con un nombre único
            img_save_path = os.path.join(label_folder, f"cell_{idx}_img.tif")
            tifffile.imwrite(img_save_path, masked_img)

            if idx % 10 == 0:
                print(f"Guardando: {img_save_path}")
            idx += 1

    print(f"Parches guardados correctamente en: {output_dir}")

# Rutas de las imágenes de entrenamiento
dataset_train = ImageDataset(
    image_paths=["/content/drive/MyDrive/tesis/image_train/1.tif", "/content/drive/MyDrive/tesis/image_train/2.tif",
                 "/content/drive/MyDrive/tesis/image_train/3.tif", "/content/drive/MyDrive/tesis/image_train/4.tif"],
    class_paths=["/content/drive/MyDrive/tesis/masks_train/1.tif", "/content/drive/MyDrive/tesis/masks_train/2.tif",
                 "/content/drive/MyDrive/tesis/masks_train/3.tif", "/content/drive/MyDrive/tesis/masks_train/4.tif"],
    labels_paths=["/content/drive/MyDrive/tesis/labels_train/1.tif", "/content/drive/MyDrive/tesis/labels_train/2.tif",
                  "/content/drive/MyDrive/tesis/labels_train/3.tif", "/content/drive/MyDrive/tesis/labels_train/4.tif"]
)

# Rutas de las imágenes de prueba
dataset_test = ImageDataset(
    image_paths=["/content/drive/MyDrive/tesis/image_test/6.tif"],
    class_paths=["/content/drive/MyDrive/tesis/masks_test/6.tif"],
    labels_paths=["/content/drive/MyDrive/tesis/labels_test/6.tif"]
)

# Procesar las imágenes de train
for i in range(len(dataset_train)):
    image_train = dataset_train.load_image(i)
    labels_train = dataset_train.load_labels(i)
    classes_train = dataset_train.load_class(i)

    save_bbox_patches(
        image_train,
        labels_train,
        classes_train,
        output_dir=f"/content/drive/MyDrive/tesis/train_patches/",
        border_margin=3
    )

# Procesar las imágenes de test
for i in range(len(dataset_test)):
    image_test = dataset_test.load_image(i)
    labels_test = dataset_test.load_labels(i)
    classes_test = dataset_test.load_class(i)

    save_bbox_patches(
        image_test,
        labels_test,
        classes_test,
        output_dir=f"/content/drive/MyDrive/tesis/test_patches/",
        border_margin=3
    )


Procesando clase: 1
Guardando: /content/drive/MyDrive/tesis/train_patches/label_1/cell_0_img.tif
Guardando: /content/drive/MyDrive/tesis/train_patches/label_2/cell_10_img.tif
Guardando: /content/drive/MyDrive/tesis/train_patches/label_3/cell_20_img.tif
Guardando: /content/drive/MyDrive/tesis/train_patches/label_1/cell_30_img.tif
Guardando: /content/drive/MyDrive/tesis/train_patches/label_4/cell_40_img.tif
Guardando: /content/drive/MyDrive/tesis/train_patches/label_1/cell_50_img.tif
Guardando: /content/drive/MyDrive/tesis/train_patches/label_4/cell_60_img.tif
Guardando: /content/drive/MyDrive/tesis/train_patches/label_1/cell_70_img.tif
Guardando: /content/drive/MyDrive/tesis/train_patches/label_1/cell_80_img.tif
Guardando: /content/drive/MyDrive/tesis/train_patches/label_4/cell_90_img.tif
Guardando: /content/drive/MyDrive/tesis/train_patches/label_1/cell_100_img.tif
Guardando: /content/drive/MyDrive/tesis/train_patches/label_3/cell_110_img.tif
Guardando: /content/drive/MyDrive/tesis/tra

OSError: [Errno 5] Input/output error: '/content/drive/MyDrive/tesis/train_patches/label_4/cell_3079_img.tif'